In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
import datetime as d
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.arima.model import ARIMA

# Define date range
s = d.datetime(2025, 1, 1)
e = d.datetime(2025, 7, 10)

# Reusable function to check stationarity
def check_stationarity(timeseries, label="Series"):
    result = adfuller(timeseries.dropna())
    print(f"\nStationarity Check for {label}:")
    print(f"ADF Statistic: {result[0]}")
    print(f"P-value: {result[1]}")
    if result[1] <= 0.05:
        print("The series is stationary.")
    else:
        print("The series is NOT stationary.")

# Function to run ARIMA analysis and forecast
def arima_analysis(stock_symbol, label):
    print(f"\n--- Analyzing {label} ({stock_symbol}) ---")
    df = yf.download(stock_symbol, start=s, end=e, auto_adjust=False)
    df.columns = df.columns.get_level_values(0)
    df = df.reset_index()
    df = df[['Close']].copy()
    df.dropna(inplace=True)

    # Check stationarity of Close
    check_stationarity(df['Close'], f"{label} Close")

    # Differencing
    df['Close_Diff'] = df['Close'].diff()
    check_stationarity(df['Close_Diff'], f"{label} Close_Diff")

    # ARIMA model
    model = ARIMA(df['Close'], order=(5, 1, 0))
    model_fit = model.fit()

    # Forecast next 10 business days
    forecast = model_fit.forecast(steps=10)
    future_dates = pd.date_range(start=df['Close'].index[-1], periods=11, freq='B')[1:]

    forecast_df = pd.DataFrame({
        'Date': future_dates,
        f'{label}_Forecast': forecast.values
    })
    print(forecast_df)
    return forecast_df

# Run for TCS, Wipro, and Infosys
tcs_forecast = arima_analysis('TCS.NS', 'TCS')
wipro_forecast = arima_analysis('WIPRO.NS', 'Wipro')
infosys_forecast = arima_analysis('INFY.NS', 'Infosys')


Dropdown(description='Stock:', options=('TCS', 'Wipro', 'Infosys', 'HCL Technologies', 'Tech Mahindra', 'HDFC …

DatePicker(value=datetime.date(2022, 1, 1), description='Start Date')

DatePicker(value=datetime.date(2025, 7, 10), description='End Date')

IntSlider(value=10, description='Forecast Days', max=60, min=5, step=5)

Button(button_style='success', description='Run Analysis', style=ButtonStyle())

Output()